## 0. Create Spark session

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext

os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--packages org.apache.hadoop:hadoop-aws:3.2.0,org.apache.hadoop:hadoop-common:3.2.0 '
    'pyspark-shell'
)
os.environ['S3_ENDPOINT'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"

if 'spark' in globals():
    spark.stop()
    print("Spark session stopped.")
else:
    print("No Spark session to stop.")

spark = (
    SparkSession.builder.master("spark://spark:7077")
    .appName("notebook-velib-to-siler")
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID"))
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY"))
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("S3_ENDPOINT"))
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000")
    .config("spark.hadoop.fs.s3a.connection.timeout", "10000")
    .getOrCreate()
)

No Spark session to stop.


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d64f1b9b-9bfd-403d-9815-39ad92c35ef5;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found org.apache.hadoop#hadoop-common;3.2.0 in central
	found org.apache.hadoop#hadoop-annotations;3.2.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.11 in c

## 1. Extract from object storage

In [2]:
os.environ['S3_INPUT_PATH'] = "s3a://velib/bronze/velib-disponibilite-en-temps-reel/*/*"

spark.sparkContext.setLogLevel("WARN")

df = spark.read.json(os.getenv("S3_INPUT_PATH"))
df.printSchema()
print(f'COUNT = {df.count()}')
df.limit(10).toPandas()

24/10/30 02:16:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- capacity: long (nullable = true)
 |-- code_insee_commune: string (nullable = true)
 |-- coordonnees_geo: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lon: double (nullable = true)
 |-- duedate: string (nullable = true)
 |-- ebike: long (nullable = true)
 |-- is_installed: string (nullable = true)
 |-- is_renting: string (nullable = true)
 |-- is_returning: string (nullable = true)
 |-- mechanical: long (nullable = true)
 |-- name: string (nullable = true)
 |-- nom_arrondissement_communes: string (nullable = true)
 |-- numbikesavailable: long (nullable = true)
 |-- numdocksavailable: long (nullable = true)
 |-- stationcode: string (nullable = true)



COUNT = 1144890


,capacity,code_insee_commune,coordonnees_geo,duedate,ebike,is_installed,is_renting,is_returning,mechanical,name,nom_arrondissement_communes,numbikesavailable,numdocksavailable,stationcode
0,21,75056,"(48.87929591733507, 2.3373600840568547)",2024-10-30T01:08:37+00:00,1,OUI,OUI,OUI,3,Toudouze - Clauzel,Paris,4,14,9020
1,35,75056,"(48.865983, 2.275725)",2024-10-30T01:03:10+00:00,1,OUI,OUI,OUI,3,Benjamin Godard - Victor Hugo,Paris,4,31,16107
2,25,75056,"(48.837525839067, 2.3360354080796)",2024-10-30T01:12:38+00:00,2,OUI,OUI,OUI,1,Cassini - Denfert-Rochereau,Paris,3,18,14111
3,20,94081,"(48.778192750803, 2.3963020229163)",2024-10-30T01:09:51+00:00,12,OUI,OUI,OUI,7,Rouget de L'isle - Watteau,Vitry-sur-Seine,19,0,44015
4,60,75056,"(48.819428333369, 2.3433353751898)",2024-10-30T01:07:55+00:00,8,OUI,OUI,OUI,9,Jourdan - Stade Charléty,Paris,17,43,14014
5,22,93066,"(48.93626891059109, 2.3588666820200914)",2024-10-30T01:12:53+00:00,7,OUI,OUI,OUI,5,Basilique,Saint-Denis,12,9,32017
6,20,75056,"(48.855907555969, 2.3925706744194)",2024-10-30T01:11:47+00:00,3,OUI,OUI,OUI,9,Charonne - Robert et Sonia Delaunay,Paris,12,8,11104
7,12,75056,"(48.875448033960744, 2.315508019010038)",2024-10-30T01:10:05+00:00,1,OUI,OUI,OUI,3,Messine - Place Du Pérou,Paris,4,7,8026
8,48,75056,"(48.835092787824, 2.3534681351338)",2024-10-30T01:10:53+00:00,2,OUI,OUI,OUI,4,Le Brun - Gobelins,Paris,6,40,13007
9,21,75056,"(48.85165383178419, 2.3308077827095985)",2024-10-30T01:07:54+00:00,0,OUI,OUI,OUI,7,Saint-Sulpice,Paris,7,14,6003


## 2. Transform

In [3]:
import pyspark.sql.functions as F

df_transformed = (
    df
    #
    .filter(F.col("is_installed") == "OUI")
    #
    .withColumn("fill_ratio", F.round(F.col("numbikesavailable") / F.col("capacity"), 3))
    .withColumn("fill_percentage", F.col("fill_ratio") * 100)
    #
    .withColumn("duedate_timestamp_minute", F.to_timestamp(
        F.concat(
            F.substring("duedate", 1, 16),      # Extracts the date and hour-minute part: "2024-10-29T01:07"
            F.lit(":00"),                       # Sets seconds explicitly to "00"
            F.substring("duedate", 20, 6)       # Extracts the timezone offset: "+00:00"
        ),
        "yyyy-MM-dd'T'HH:mm:ssXXX"
    ))
    .withColumn("polldate_timestamp_minute", 
        F.to_timestamp(
            F.split(F.split(F.input_file_name(), "/")[6], "\.")[0], 
            "yyyy-MM-dd-HHmm"
        )
    )
    .withColumn("polldate", F.concat(
        F.date_format(F.col("polldate_timestamp_minute"), "yyyy-MM-dd'T'HH:mm:ss"),
        F.lit("+00:00")
    ))
    #
    .withColumn("lat", F.col("coordonnees_geo.lat"))
    .withColumn("lon", F.col("coordonnees_geo.lon"))
    .drop("coordonnees_geo")
    #
    .withColumn("part_month", F.substring(F.col("polldate"), 1, 7))
    .withColumn("part_day", F.substring(F.col("polldate"), 1, 10))
    .withColumn("part_minute", F.substring(F.col("polldate"), 1, 16))
    .filter(F.col("part_day") >= "2024-01-01")
)
df_transformed = df_transformed.select(*(sorted(df_transformed.columns)))
df_transformed.printSchema()
print(f'COUNT = {df_transformed.count()}')

import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)
df_transformed.limit(10).toPandas()

root
 |-- capacity: long (nullable = true)
 |-- code_insee_commune: string (nullable = true)
 |-- duedate: string (nullable = true)
 |-- duedate_timestamp_minute: timestamp (nullable = true)
 |-- ebike: long (nullable = true)
 |-- fill_percentage: double (nullable = true)
 |-- fill_ratio: double (nullable = true)
 |-- is_installed: string (nullable = true)
 |-- is_renting: string (nullable = true)
 |-- is_returning: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- mechanical: long (nullable = true)
 |-- name: string (nullable = true)
 |-- nom_arrondissement_communes: string (nullable = true)
 |-- numbikesavailable: long (nullable = true)
 |-- numdocksavailable: long (nullable = true)
 |-- part_day: string (nullable = true)
 |-- part_minute: string (nullable = true)
 |-- part_month: string (nullable = true)
 |-- polldate: string (nullable = true)
 |-- polldate_timestamp_minute: timestamp (nullable = true)
 |-- stationcode: string (nulla

COUNT = 1123302


,capacity,code_insee_commune,duedate,duedate_timestamp_minute,ebike,fill_percentage,fill_ratio,is_installed,is_renting,is_returning,lat,lon,mechanical,name,nom_arrondissement_communes,numbikesavailable,numdocksavailable,part_day,part_minute,part_month,polldate,polldate_timestamp_minute,stationcode
0,21,75056,2024-10-30T01:08:37+00:00,2024-10-30 01:08:00,1,19.0,0.190,OUI,OUI,OUI,48.879296,2.337360,3,Toudouze - Clauzel,Paris,4,14,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,9020
1,35,75056,2024-10-30T01:03:10+00:00,2024-10-30 01:03:00,1,11.4,0.114,OUI,OUI,OUI,48.865983,2.275725,3,Benjamin Godard - Victor Hugo,Paris,4,31,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,16107
2,25,75056,2024-10-30T01:12:38+00:00,2024-10-30 01:12:00,2,12.0,0.120,OUI,OUI,OUI,48.837526,2.336035,1,Cassini - Denfert-Rochereau,Paris,3,18,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,14111
3,20,94081,2024-10-30T01:09:51+00:00,2024-10-30 01:09:00,12,95.0,0.950,OUI,OUI,OUI,48.778193,2.396302,7,Rouget de L'isle - Watteau,Vitry-sur-Seine,19,0,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,44015
4,60,75056,2024-10-30T01:07:55+00:00,2024-10-30 01:07:00,8,28.3,0.283,OUI,OUI,OUI,48.819428,2.343335,9,Jourdan - Stade Charléty,Paris,17,43,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,14014
5,22,93066,2024-10-30T01:12:53+00:00,2024-10-30 01:12:00,7,54.5,0.545,OUI,OUI,OUI,48.936269,2.358867,5,Basilique,Saint-Denis,12,9,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,32017
6,20,75056,2024-10-30T01:11:47+00:00,2024-10-30 01:11:00,3,60.0,0.600,OUI,OUI,OUI,48.855908,2.392571,9,Charonne - Robert et Sonia Delaunay,Paris,12,8,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,11104
7,12,75056,2024-10-30T01:10:05+00:00,2024-10-30 01:10:00,1,33.3,0.333,OUI,OUI,OUI,48.875448,2.315508,3,Messine - Place Du Pérou,Paris,4,7,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,8026
8,48,75056,2024-10-30T01:10:53+00:00,2024-10-30 01:10:00,2,12.5,0.125,OUI,OUI,OUI,48.835093,2.353468,4,Le Brun - Gobelins,Paris,6,40,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,13007
9,21,75056,2024-10-30T01:07:54+00:00,2024-10-30 01:07:00,0,33.3,0.333,OUI,OUI,OUI,48.851654,2.330808,7,Saint-Sulpice,Paris,7,14,2024-10-30,2024-10-30T02:13,2024-10,2024-10-30T02:13:00+00:00,2024-10-30 02:13:00,6003


## 3. Load into object storage

In [4]:
os.environ['S3_OUTPUT_PATH'] = "s3a://velib/silver/velib-disponibilite-en-temps-reel"

spark.sparkContext.setLogLevel("WARN")

# Write DataFrame to S3
(
    df_transformed.write
    .partitionBy("part_day")
    .format("parquet")
    .mode("overwrite")
    .save(os.getenv('S3_OUTPUT_PATH'))
)

## 4. Update hive metastore

In [5]:
!pip install trino

In [6]:
from pyspark.sql import DataFrame

def generate_trino_create_table(
    df: DataFrame, 
    catalog: str, 
    schema: str, 
    table: str, 
    partitioned_by: str, 
    external_location: str
) -> str:
    # Comprehensive mapping of Spark SQL types to Trino types
    type_mapping = {
        "bigint": "BIGINT",
        "binary": "VARBINARY",
        "boolean": "BOOLEAN",
        "decimal": "DECIMAL",  # Precision and scale will need to be handled separately if defined
        "double": "DOUBLE",
        "float": "REAL",
        "int": "INTEGER",
        "smallint": "SMALLINT",
        "string": "VARCHAR",
        "timestamp": "TIMESTAMP",
        "tinyint": "TINYINT"
    }
    
    # Separate columns to ensure partition column is placed last
    columns = []
    partition_column = None
    for field in df.schema.fields:
        spark_type = field.dataType.simpleString()
        trino_type = type_mapping.get(spark_type, "VARCHAR")  # default to VARCHAR if no match
        # DEBUG # print(f'field.name={field.name}, spark_type={spark_type}, trino_type={trino_type}')
        if field.name == partitioned_by:
            partition_column = f"{field.name} {trino_type}"
        else:
            columns.append(f"{field.name} {trino_type}")
    
    # Add the partition column to the end if it exists in the schema
    if partition_column:
        columns.append(partition_column)
    
    # Join column definitions into a single string
    columns_definition = ",\n    ".join(columns)
    
    # Generate the final CREATE TABLE statement
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {catalog}.{schema}.{table} (
        {columns_definition}
    ) 
    WITH (
        format = 'PARQUET',
        partitioned_by = ARRAY['{partitioned_by}'],
        external_location = '{external_location}'
    )
    """
    
    return create_table_sql.strip()


In [7]:
import trino

host, port, user = 'trino-coordinator', 8080, 'trino'
conn = trino.dbapi.connect(host=host, port=port, user=user)
cur = conn.cursor()

catalog, schema, table = 'minio', 'velib_silver', 'velib_disponibilite_en_temps_reel'
schema_location = 's3a://velib/silver/'
partitioned_by = 'part_day'
external_location = 's3a://velib/silver/velib-disponibilite-en-temps-reel'

# List of queries to execute
queries = [
    f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema} WITH (location = '{schema_location}')",
    f"DROP TABLE IF EXISTS {catalog}.{schema}.{table}",
    
    generate_trino_create_table(df_transformed, catalog, schema, table, partitioned_by, external_location),
    
    f"USE {catalog}.{schema}",
    f"CALL system.sync_partition_metadata('{schema}', '{table}', 'ADD')",
    f"SELECT * FROM {catalog}.{schema}.{table} LIMIT 5"
]

# Execute each query in the list
for query in queries:
    try:
        cur.execute(query)
        # Check if the query is a SELECT query to fetch results
        if query.startswith("SELECT"):
            results = cur.fetchall()
            for row in results:
                print(row)
        else:
            print(f"Executed: {query}")
    except Exception as e:
        print(f"Error executing query: {query}. Error: {e}")

# Close the cursor and connection
cur.close()
conn.close()


Executed: CREATE SCHEMA IF NOT EXISTS minio.velib_silver WITH (location = 's3a://velib/silver/')
Executed: DROP TABLE IF EXISTS minio.velib_silver.velib_disponibilite_en_temps_reel
Executed: CREATE TABLE IF NOT EXISTS minio.velib_silver.velib_disponibilite_en_temps_reel (
        capacity BIGINT,
    code_insee_commune VARCHAR,
    duedate VARCHAR,
    duedate_timestamp_minute TIMESTAMP,
    ebike BIGINT,
    fill_percentage DOUBLE,
    fill_ratio DOUBLE,
    is_installed VARCHAR,
    is_renting VARCHAR,
    is_returning VARCHAR,
    lat DOUBLE,
    lon DOUBLE,
    mechanical BIGINT,
    name VARCHAR,
    nom_arrondissement_communes VARCHAR,
    numbikesavailable BIGINT,
    numdocksavailable BIGINT,
    part_minute VARCHAR,
    part_month VARCHAR,
    polldate VARCHAR,
    polldate_timestamp_minute TIMESTAMP,
    stationcode VARCHAR,
    part_day VARCHAR
    ) 
    WITH (
        format = 'PARQUET',
        partitioned_by = ARRAY['part_day'],
        external_location = 's3a://velib/s

24/10/30 02:21:29 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/10/30 02:21:29 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:873)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$Mess